In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 48.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=3d08e443fec5f40455fdcde595735446c17ab523a3052e38896f81c5c07a3761
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz
!pip install -q findspark


In [ ]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf




In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('PDC-project-cooler') \
    .getOrCreate()

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/soc-LiveJournal1.txt', delimiter='\t')

In [ ]:
data.head()

,FromNodeId,ToNodeId
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [ ]:
data.columns = ['User','Connections']

In [ ]:
data.head()

,User,Connections
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [ ]:
data.to_csv('prepared_data.csv')


In [ ]:
network_data1 = spark.read.option('header','true').csv('prepared_data.csv',inferSchema=True)

In [ ]:
network_data1.head(5)

[Row(_c0=0, User=0, Connections=1),
 Row(_c0=1, User=0, Connections=2),
 Row(_c0=2, User=0, Connections=3),
 Row(_c0=3, User=0, Connections=4),
 Row(_c0=4, User=0, Connections=5)]

In [ ]:
column_to_drop = ['_c0']
network_data1 = network_data1.drop(*column_to_drop)

In [ ]:
network_data1.head(5)

[Row(User=0, Connections=1),
 Row(User=0, Connections=2),
 Row(User=0, Connections=3),
 Row(User=0, Connections=4),
 Row(User=0, Connections=5)]

In [ ]:
network_data1.na.drop(how='any')

DataFrame[User: int, Connections: int]

In [ ]:
network_data1.printSchema()

root
 |-- User: integer (nullable = true)
 |-- Connections: integer (nullable = true)



In [ ]:
network_data = spark.read.csv('prepared_data.csv', inferSchema=True, header=True).rdd

In [ ]:
no_connections = network_data.map(lambda row: (row[1], 1)).reduceByKey(lambda a,b: a+b)

In [ ]:
connections_list_distinct = no_connections.collect()
network_data = 0
network_data2 = 0
data=0

In [ ]:
connections_list=list()
connections_list_connections = list()
common_ = list()

connections = network_data1.select('Connections').collect()

for j in connections:
  if j[0]<5:
    connections_list_connections.append(connections_list_distinct[j[0]][1])
  else:
    connections_list_connections.append(0)
connections=0

users = network_data1.select('User').collect()
for i in users:
  connections_list.append(connections_list_distinct[i[0]][1])
users=0

distinct_users = network_data1.select('User').distinct().collect()
for i in distinct_users:
  specific_user = 0
  specific_user = network_data1.select('Connections').where(network_data1['User']==i[0]).collect()
  for j in specific_user:
    common_.append([i[0],j[0]])

In [ ]:
no_connections=0
common_.sort()


In [ ]:
k=0
common_neighbours=list()
for i, j in enumerate(common_):
  
  if connections_list[j[0]]==0 or connections_list_connections[i]==0:
    k=0 
    common_neighbours.append(k)
  else:
    B = network_data1.select('Connections').where(network_data1['User']==j[0])

    if j[0]!=j[1]:
      A = network_data1.select('Connections').where(network_data1['User']==j[1])

    k=A.select("Connections").distinct().join((B.select("Connections").distinct()),\
                                                ['Connections'], how='inner').select('Connections').count()
    common_neighbours.append(k)

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window

network_data_temp = network_data1
b = spark.createDataFrame([(l,) for l in common_neighbours], ['CommonNeighbours'])

#add 'sequential' index and join both dataframe to get the final result
network_data_temp = network_data_temp.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

network_data_temp = network_data_temp.join(b, network_data_temp.row_idx == b.row_idx).\
             drop("row_idx")

b = spark.createDataFrame([(l,) for l in connections_list], ['NoUserConnections'])

network_data_temp = network_data_temp.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

network_data_temp = network_data_temp.join(b, network_data_temp.row_idx == b.row_idx).\
             drop("row_idx")

b = spark.createDataFrame([(l,) for l in connections_list_connections], ['NoConnectionsConnections'])

network_data_temp = network_data_temp.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
b = b.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

network_data_temp = network_data_temp.join(b, network_data_temp.row_idx == b.row_idx).\
             drop("row_idx")


network_data_temp.show()

+----+-----------+----------------+-----------------+------------------------+
|User|Connections|CommonNeighbours|NoUserConnections|NoConnectionsConnections|
+----+-----------+----------------+-----------------+------------------------+
|   0|          1|               3|               46|                     210|
|   0|          2|               0|               46|                      97|
|   0|          3|               3|               46|                     102|
|   0|          4|               5|               46|                      71|
|   0|          5|               0|               46|                       0|
|   0|          6|               0|               46|                       0|
|   0|          7|               0|               46|                       0|
|   0|          8|               0|               46|                       0|
|   0|          9|               0|               46|                       0|
|   0|         10|               0|               46

In [ ]:
network_data_temp = network_data_temp.drop('_c0')

In [ ]:
network_data_temp.show(50)

+----+-----------+----------------+-----------------+------------------------+
|User|Connections|CommonNeighbours|NoUserConnections|NoConnectionsConnections|
+----+-----------+----------------+-----------------+------------------------+
|   0|          1|               3|               46|                     210|
|   0|          2|               0|               46|                      97|
|   0|          3|               3|               46|                     102|
|   0|          4|               5|               46|                      71|
|   0|          5|               0|               46|                       0|
|   0|          6|               0|               46|                       0|
|   0|          7|               0|               46|                       0|
|   0|          8|               0|               46|                       0|
|   0|          9|               0|               46|                       0|
|   0|         10|               0|               46

In [ ]:
network_data1 = network_data_temp

In [ ]:
def findCN(user,connection):
  k=0
  A=[]
  B=[]
  B = network_data1.select('Connections').where(network_data1['User']==user)
  if user!=connection:
    A = network_data1.select('Connections').where(network_data1['User']==connection)
  else:
    return 0

  k=A.select("Connections").distinct().join((B.select("Connections").distinct()),\
  ['Connections'], how='inner').select('Connections').count()
  return k

In [ ]:

import random
from pyspark.sql.functions import col
from pyspark.sql.functions import when
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number

randomlist=list(list())
connections_of_user=list()

# print(connections_list_distinct)
for connection in connections_list_distinct:
  connections_of_user.append(int((connection[1]**2)/2))

# print(connections_of_user)
for connection in connections_of_user:
  randomlist.append(random.sample(range(0,6500000),connection))

NotConnected=list(list())
no_connect=0
cn=1
count_connected=network_data1.count()
network_data1=network_data1.withColumn("row_num",F.row_number().over(Window.partitionBy(network_data1['User']).orderBy(network_data1['User'])))


network_data1.createOrReplaceTempView('Network')
 
#Adding every user's non connection

for i in range(len(randomlist)):

   for j in range(len(randomlist[i])):

      rownum=count_connected+1+(i*len(randomlist[i])+j)
      if randomlist[i][j]>len(connections_list_distinct):
        no_connect=0
        NotConnected.append([i,randomlist[i][j],0,connections_list_distinct[i][1],no_connect,rownum])
      else:
        no_connect=connections_list_distinct[randomlist[i][j]][1]
        # print(i,randomlist[i][j])
        NotConnected.append([i,randomlist[i][j],findCN(i,randomlist[i][j]),connections_list_distinct[i][1],no_connect,rownum])
    


NotConnected_df=spark.createDataFrame(NotConnected,network_data1.columns)
count_notconnected=NotConnected_df.count()
# print(NotConnected_df.show())
# print(network_data1.show())
# print(count_notconnected)
# print(count_connected)
network_data1=network_data1.union(NotConnected_df)


print(network_data1.show())
network_data1=network_data1.dropDuplicates()
network_data1 = network_data1.sort(['User', 'Connections'])
network_data1=network_data1.withColumn("Labels",network_data1['User']-network_data1['User']+1)
network_data1=network_data1.withColumn("Labels",when(network_data1['row_num']>count_connected,0).otherwise(network_data1['Labels']))
# network_data1.show()

+----+-----------+----------------+-----------------+------------------------+-------+
|User|Connections|CommonNeighbours|NoUserConnections|NoConnectionsConnections|row_num|
+----+-----------+----------------+-----------------+------------------------+-------+
|   0|          1|               3|               46|                     210|      1|
|   0|          2|               0|               46|                      97|      2|
|   0|          3|               3|               46|                     102|      3|
|   0|          4|               5|               46|                      71|      4|
|   0|          5|               0|               46|                       0|      5|
|   0|          6|               0|               46|                       0|      6|
|   0|          7|               0|               46|                       0|      7|
|   0|          8|               0|               46|                       0|      8|
|   0|          9|               0|        

In [ ]:
network_data1 = network_data1.sort('row_num')
network_data1.show()

+----+-----------+----------------+-----------------+------------------------+-------+------+
|User|Connections|CommonNeighbours|NoUserConnections|NoConnectionsConnections|row_num|Labels|
+----+-----------+----------------+-----------------+------------------------+-------+------+
|   4|          0|               5|               71|                      46|      1|     1|
|   3|          0|               3|              102|                      46|      1|     1|
|   1|          0|               3|              210|                      46|      1|     1|
|   2|        117|               0|               97|                       0|      1|     1|
|   0|          1|               3|               46|                     210|      1|     1|
|   4|          8|               0|               71|                       0|      2|     1|
|   2|        135|               0|               97|                       0|      2|     1|
|   1|          1|               3|              210|       

In [ ]:
from pyspark.ml.feature import VectorAssembler

independentCols = ['User', 'Connections', 'CommonNeighbours', 'NoUserConnections','NoConnectionsConnections']
assembler = VectorAssembler(inputCols=independentCols, outputCol="features")
network_data1 = assembler.transform(network_data1)
network_data1.show()

+----+-----------+----------------+-----------------+------------------------+-------+------+--------------------+
|User|Connections|CommonNeighbours|NoUserConnections|NoConnectionsConnections|row_num|Labels|            features|
+----+-----------+----------------+-----------------+------------------------+-------+------+--------------------+
|   3|          0|               3|              102|                      46|      1|     1|[3.0,0.0,3.0,102....|
|   2|        117|               0|               97|                       0|      1|     1|[2.0,117.0,0.0,97...|
|   4|          0|               5|               71|                      46|      1|     1|[4.0,0.0,5.0,71.0...|
|   1|          0|               3|              210|                      46|      1|     1|[1.0,0.0,3.0,210....|
|   0|          1|               3|               46|                     210|      1|     1|[0.0,1.0,3.0,46.0...|
|   4|          8|               0|               71|                       0|  

In [ ]:
network_data_rdd=network_data1.rdd
train, test = network_data1.randomSplit([0.7, 0.3], seed = 2018)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Labels')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select("Labels", "prediction").show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Labels", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.9798911463827378
Test Error = 0.020108853617262223


In [ ]:
#